# Converting NEXRAD Level 2 Data to Zarr Format

This notebook demonstrates the core workflow for converting NEXRAD Level 2 radar data into Analysis-Ready Cloud-Optimized (ARCO) Zarr format using the `raw2zarr` library.

```{note}
This example processes NEXRAD data from the AWS Open Data Registry and converts it to hierarchical Zarr format for efficient analysis.
```

## Import Libraries

In [ ]:
import os

import xarray as xr

from raw2zarr.builder.builder_utils import get_icechunk_repo

# Import the current API - convert_files is the main entry point
from raw2zarr.builder.convert import convert_files
from raw2zarr.utils import list_nexrad_files

## Discover NEXRAD Files

We'll use the `list_nexrad_files()` utility to find NEXRAD Level 2 radar files from the AWS Open Data Registry.

In [2]:
# Configuration
radar = "KVNX"  # Oklahoma radar station
zarr_store = f"../zarr/{radar}.zarr"  # Output Zarr store path

In [3]:
# Discover NEXRAD files for May 20, 2011 (sever weather outbreak day)
radar_files = list_nexrad_files(
    radar="KVNX", start_time="2011-05-20 00:00", end_time="2011-05-20 23:59"
)

print(f"Found {len(radar_files)} NEXRAD files for {radar}")

Found 320 NEXRAD files for KVNX


## Convert to Zarr Format

Convert radar files to Zarr format using sequential processing.

In [ ]:
# Select subset for demonstration - use environment variable if available
num_files = int(os.getenv("DEMO_FILE_COUNT", 35))
start_idx = int(os.getenv("DEMO_START_INDEX", 135))
test_files = radar_files[start_idx : start_idx + num_files]

print(f"Processing {len(test_files)} files for demonstration")

Create Icechunk repository for versioned Zarr storage.

In [ ]:
# Create Icechunk repository using utility function
repo = get_icechunk_repo(zarr_store, use_manifest_config=True)

In [6]:
%%time
convert_files(
    radar_files=test_files,
    append_dim="vcp_time",
    repo=repo,
    zarr_format=3,
    engine="nexradlevel2",
    process_mode="sequential",
    remove_strings=True,
)

print(f"Conversion completed! Zarr store created at: {zarr_store}")

[icechunk] Committed s3://noaa-nexrad-level2/2011/05/20/KVNX/KVNX20110520_094849_V06.gz as snapshot TZJ2FVGRFEBHGE40TPWG
[icechunk] Committed s3://noaa-nexrad-level2/2011/05/20/KVNX/KVNX20110520_095308_V06.gz as snapshot DSRFJHGBYKMJMMKY7510
[icechunk] Committed s3://noaa-nexrad-level2/2011/05/20/KVNX/KVNX20110520_095727_V06.gz as snapshot SSZNP1D4CYG7PYH43CQ0
[icechunk] Committed s3://noaa-nexrad-level2/2011/05/20/KVNX/KVNX20110520_100147_V06.gz as snapshot YSJYBEP6YYQFEA37SF20
[icechunk] Committed s3://noaa-nexrad-level2/2011/05/20/KVNX/KVNX20110520_100608_V06.gz as snapshot JFZHFKN69ZRXBDRJWGYG
[icechunk] Committed s3://noaa-nexrad-level2/2011/05/20/KVNX/KVNX20110520_101028_V06.gz as snapshot R2AYSMZSP5YW9MFGQSB0
[icechunk] Committed s3://noaa-nexrad-level2/2011/05/20/KVNX/KVNX20110520_101447_V06.gz as snapshot MTDTWT2QDEKA0G8KE8RG
[icechunk] Committed s3://noaa-nexrad-level2/2011/05/20/KVNX/KVNX20110520_101906_V06.gz as snapshot JEJWCXRE32DQD88NB1BG
[icechunk] Committed s3://noaa-n

## Read Data from Zarr Store

Read the converted data back from the Zarr store.

Create a readonly session to access the converted data.

In [11]:
session = repo.readonly_session("main")

Open the radar datatree using xarray.

In [9]:
dtree = xr.open_datatree(session.store, engine="zarr", consolidated=False, chunks={})

In [10]:
dtree

<xarray.DataTree>
Group: /
└── Group: /VCP-12
    │   Dimensions:        (vcp_time: 35)
    │   Coordinates:
    │     * vcp_time       (vcp_time) datetime64[ns] 280B 2011-05-20T09:48:49 ... 201...
    │   Data variables:
    │       latitude       (vcp_time) float32 140B dask.array<chunksize=(1,), meta=np.ndarray>
    │       altitude       (vcp_time) float64 280B dask.array<chunksize=(1,), meta=np.ndarray>
    │       longitude      (vcp_time) float32 140B dask.array<chunksize=(1,), meta=np.ndarray>
    │       volume_number  (vcp_time) float64 280B dask.array<chunksize=(1,), meta=np.ndarray>
    │   Attributes:
    │       Conventions:      None
    │       instrument_name:  KVNX
    │       version:          None
    │       title:            None
    │       institution:      None
    │       references:       None
    │       source:           None
    │       history:          None
    │       comment:          im/exported using xradar
    │       scan_name:        VCP-12
    ├── Group: /VCP-12/georeferencing_correction
    │       Dimensions:    ()
    │       Coordinates:
    │           altitude   int64 8B ...
    │           longitude  float64 8B ...
    │           latitude   float64 8B ...
    ├── Group: /VCP-12/sweep_1
    │       Dimensions:            (vcp_time: 35, azimuth: 720, range: 1192)
    │       Coordinates:
    │         * azimuth            (azimuth) float64 6kB 0.25 0.75 1.25 ... 359.2 359.8
    │           crs_wkt            int64 8B ...
    │           longitude          float64 8B ...
    │           latitude           float64 8B ...
    │           elevation          (azimuth) float64 6kB dask.array<chunksize=(720,), meta=np.ndarray>
    │         * range              (range) float32 5kB 2.125e+03 2.375e+03 ... 2.999e+05
    │           altitude           int64 8B ...
    │           time               (azimuth) datetime64[ns] 6kB dask.array<chunksize=(720,), meta=np.ndarray>
    │           x                  (azimuth, range) float64 7MB dask.array<chunksize=(180, 298), meta=np.ndarray>
    │           z                  (azimuth, range) float64 7MB dask.array<chunksize=(180, 298), meta=np.ndarray>
    │           y                  (azimuth, range) float64 7MB dask.array<chunksize=(180, 298), meta=np.ndarray>
    │       Data variables:
    │           VRADH              (vcp_time, azimuth, range) float32 120MB dask.array<chunksize=(1, 720, 1192), meta=np.ndarray>
    │           DBZH               (vcp_time, azimuth, range) float32 120MB dask.array<chunksize=(1, 720, 1192), meta=np.ndarray>
    │           WRADH              (vcp_time, azimuth, range) float32 120MB dask.array<chunksize=(1, 720, 1192), meta=np.ndarray>
    │           sweep_number       (vcp_time) float64 280B dask.array<chunksize=(1,), meta=np.ndarray>
    │           sweep_fixed_angle  (vcp_time) float32 140B dask.array<chunksize=(1,), meta=np.ndarray>
    ├── Group: /VCP-12/sweep_12
    │       Dimensions:            (vcp_time: 35, azimuth: 360, range: 540)
    │       Coordinates:
    │         * azimuth            (azimuth) float64 3kB 0.5 1.5 2.5 ... 357.5 358.5 359.5
    │           latitude           float64 8B ...
    │           altitude           int64 8B ...
    │           elevation          (azimuth) float64 3kB dask.array<chunksize=(360,), meta=np.ndarray>
    │           crs_wkt            int64 8B ...
    │         * range              (range) float32 2kB 2.125e+03 2.375e+03 ... 1.369e+05
    │           y                  (azimuth, range) float64 2MB dask.array<chunksize=(180, 270), meta=np.ndarray>
    │           x                  (azimuth, range) float64 2MB dask.array<chunksize=(180, 270), meta=np.ndarray>
    │           time               (azimuth) datetime64[ns] 3kB dask.array<chunksize=(360,), meta=np.ndarray>
    │           longitude          float64 8B ...
    │           z                  (azimuth, range) float64 2MB dask.array<chunksize=(180, 270), meta=np.ndarray>
    │       Data variables:
 

The datatree contains VCP nodes organized by Volume Coverage Pattern.

In [14]:
list(dtree.children)

['VCP-12']

Navigate the radar datatree using path syntax like `VCP-12/sweep_0/DBZH`.

In [13]:
dtree["VCP-12/sweep_0/DBZH"]

<xarray.DataArray 'DBZH' (vcp_time: 35, azimuth: 720, range: 1832)> Size: 185MB
dask.array<open_dataset-DBZH, shape=(35, 720, 1832), dtype=float32, chunksize=(1, 720, 1832), chunktype=numpy.ndarray>
Coordinates:
  * vcp_time   (vcp_time) datetime64[ns] 280B 2011-05-20T09:48:49 ... 2011-05...
    altitude   int64 8B ...
    latitude   float64 8B ...
    longitude  float64 8B ...
  * azimuth    (azimuth) float64 6kB 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
    elevation  (azimuth) float64 6kB dask.array<chunksize=(720,), meta=np.ndarray>
    crs_wkt    int64 8B ...
    time       (azimuth) datetime64[ns] 6kB dask.array<chunksize=(720,), meta=np.ndarray>
    x          (azimuth, range) float64 11MB dask.array<chunksize=(180, 458), meta=np.ndarray>
    y          (azimuth, range) float64 11MB dask.array<chunksize=(180, 458), meta=np.ndarray>
  * range      (range) float32 7kB 2.125e+03 2.375e+03 ... 4.596e+05 4.599e+05
    z          (azimuth, range) float64 11MB dask.array<chunksize=(180, 458), meta=np.ndarray>
Attributes:
    long_name:      Equivalent reflectivity factor H
    units:          dBZ
    standard_name:  radar_equivalent_reflectivity_factor_h

## Summary

This notebook demonstrated the core workflow for converting NEXRAD radar data to Zarr format:

1. **Discovery**: Used `list_nexrad_files()` to find radar files from AWS S3
2. **Conversion**: Applied `convert_files()` with sequential processing for simplicity  
3. **Access**: Read the hierarchical Zarr data back using xarray DataTree

The resulting Zarr store organizes radar data by Volume Coverage Pattern (VCP) and elevation sweeps, making it easy to access specific time periods and elevations for analysis.